In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import os
import uuid
from keras.applications.efficientnet import EfficientNetB1
from PIL import Image
from tqdm import tqdm
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms, io
import time
import copy
from torchvision.models import efficientnet_b1,EfficientNet_B1_Weights
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import roc_auc_score
import json
import pathlib
from torch.nn.functional import softmax
from torch.utils.data import DataLoader

# Preprocessing data

In [5]:
# Set the paths to your data folders
data_folder = './dont need now/CASIA_thousand_norm_256_64_e_nn_stacked'

folders=['train', 'test', 'val']

if not os.path.isdir(os.path.join("./closed_set")):
              os.mkdir(os.path.join("./closed_set"))

def combine_images(folder):
    for fol in folders:
      folder_combine=os.path.join(folder,fol)
      user_folders_L = [file for file in os.listdir(folder_combine) if file.endswith(('_L'))]
      user_folders_R = [file for file in os.listdir(folder_combine) if file.endswith(('_R'))]
      for L_folders in tqdm(user_folders_L):
        folder_combine_L=os.path.join(folder_combine,L_folders)
        images_L = [file for file in os.listdir(folder_combine_L) if file.endswith(('.png', '.jpg'))]
        for image in images_L:
          image_path = os.path.join(folder_combine_L,image)
          im = Image.open(image_path)
          path_to_save_folder = os.path.join("./closed_set", L_folders)
          if not os.path.isdir(path_to_save_folder):
              os.mkdir(path_to_save_folder)
          im.save(os.path.join(path_to_save_folder, f'{uuid.uuid4().hex}.png'))

      for R_folders in tqdm(user_folders_R):
        folder_combine_R=os.path.join(folder_combine,R_folders)
        images_R = [file for file in os.listdir(folder_combine_R) if file.endswith(('.png', '.jpg'))]
        for image in images_R:
          image_path = os.path.join(folder_combine_R,image)
          im = Image.open(image_path)
          path_to_save_folder = os.path.join("./closed_set", R_folders)
          if not os.path.isdir(path_to_save_folder):
              os.mkdir(path_to_save_folder)
          im.save(os.path.join(path_to_save_folder, f'{uuid.uuid4().hex}.png'))

In [ ]:
combine_images(data_folder)

In [7]:
# check if there are folders with a len(folder)!=10
data_folder2 = './closed_set'
files_1=os.listdir(data_folder2)
print(len(files_1))

for i in files_1:
  ff=os.path.join(data_folder2,i)
  if len(os.listdir(ff))!=10:
    print(len(os.listdir(ff)))

3
7
7
3


In [ ]:
data_folder2 = './closed_set'
user_numbers = [file.split('_')[0] for file in os.listdir(data_folder2) if file.endswith(('_L'))]
if not os.path.isdir(os.path.join("./stage_1_data")):
  os.mkdir(os.path.join("./stage_1_data"))

for un in tqdm(user_numbers):
  LL=os.path.join(data_folder2,un+'_L')
  RR=os.path.join(data_folder2,un+'_R')
  images_LL=os.listdir(LL)
  images_RR=os.listdir(RR)

  if len(images_LL)>len(images_RR):
    for val,ii in enumerate(images_RR):
      im_cropped_R=Image.open(os.path.join(RR,ii)).crop((0,0,256,128))
      im_cropped_L=Image.open(os.path.join(LL,images_LL[val])).crop((0,0,256,128))
      new_im=Image.new('RGB',(256,256))
      new_im.paste(im_cropped_L, (0, 0))
      new_im.paste(im_cropped_R, (0, 128))
      path_to_save_dt = os.path.join("./stage_1_data", un)
      if not os.path.isdir(path_to_save_dt):
        os.mkdir(path_to_save_dt)
      new_im.save(os.path.join(path_to_save_dt, f'{uuid.uuid4().hex}.png'))
  else:
    for val,ii in enumerate(images_LL):
      im_cropped_L=Image.open(os.path.join(LL,ii)).crop((0,0,256,128))
      im_cropped_R=Image.open(os.path.join(RR,images_RR[val])).crop((0,0,256,128))
      new_im=Image.new('RGB',(256,256))
      new_im.paste(im_cropped_L, (0, 0))
      new_im.paste(im_cropped_R, (0, 128))
      path_to_save_dt = os.path.join("./stage_1_data", un)
      if not os.path.isdir(path_to_save_dt):
        os.mkdir(path_to_save_dt)
      new_im.save(os.path.join(path_to_save_dt, f'{uuid.uuid4().hex}.png'))


In [ ]:
data_folder3="./stage_1_data"
idds=os.listdir(data_folder3)
print(len(idds))
for i in idds:
  ii=os.path.join(data_folder3,i)
  if len(os.listdir(ii))!=10:
    print(len(os.listdir(ii)))

250
9
8
9
9


In [ ]:
data_folder3="./stage_1_data"
idds=os.listdir(data_folder3)
if not os.path.isdir(os.path.join("./CASIA_1")):
    os.mkdir(os.path.join("./CASIA_1"))
if not os.path.isdir(os.path.join("./CASIA_1", 'train')):
    os.mkdir(os.path.join("./CASIA_1", 'train'))
if not os.path.isdir(os.path.join("./CASIA_1", 'test')):
    os.mkdir(os.path.join("./CASIA_1", 'test'))
if not os.path.isdir(os.path.join("./CASIA_1", 'val')):
    os.mkdir(os.path.join("./CASIA_1", 'val'))

for i in tqdm(idds):
  ii=os.path.join(data_folder3,i)
  iii=os.listdir(ii)
  path_to_save_train_dt = os.path.join("./CASIA_1", 'train', i)
  if not os.path.isdir(path_to_save_train_dt):
    os.mkdir(path_to_save_train_dt)
  path_to_save_test_dt = os.path.join("./CASIA_1", 'test', i)
  if not os.path.isdir(path_to_save_test_dt):
    os.mkdir(path_to_save_test_dt)
  path_to_save_val_dt = os.path.join("./CASIA_1", 'val', i)
  if not os.path.isdir(path_to_save_val_dt):
    os.mkdir(path_to_save_val_dt)
  if len(iii)==7:
    tr_dt=iii[:5]
    vl_dt=iii[5:6]
    tst_dt=iii[6:]
  elif len(iii)==8:
    tr_dt=iii[:6]
    vl_dt=iii[6:7]
    tst_dt=iii[7:]
  elif len(iii)==9:
    tr_dt=iii[:7]
    vl_dt=iii[7:8]
    tst_dt=iii[8:]
  else:
    tr_dt=iii[:7]
    vl_dt=iii[7:9]
    tst_dt=iii[9:]

  for tr_i in tr_dt:
    im=Image.open(os.path.join(ii,tr_i))
    im.save(os.path.join(path_to_save_train_dt, f'{uuid.uuid4().hex}.png'))
  for vl_i in vl_dt:
    im=Image.open(os.path.join(ii,vl_i))
    im.save(os.path.join(path_to_save_val_dt, f'{uuid.uuid4().hex}.png'))
  for tst_i in tst_dt:
    im=Image.open(os.path.join(ii,tst_i))
    im.save(os.path.join(path_to_save_test_dt, f'{uuid.uuid4().hex}.png'))
print('')
print('fnshd')

100%|██████████| 250/250 [02:01<00:00,  2.05it/s]


fnshd
